# GoogleNet简单实现

## 准备数据集

In [3]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

batch_size = 64
# 使用torch把图像转化为tensor,图像 c * w * h
# 把单通道转换成多通道，把一系列的图像进行转换，变成 [0,1]之间。0.1307是均值，0.3081是标准差
# 使用Normalize也是希望像素值满足 0 1 分布
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])
# 变换好之后直接放进数据集里
train_dataset = datasets.MNIST(root='./dataset/mnist',train=True,download=True,transform=transform)
train_loader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
test_dataset = datasets.MNIST(root='./dataset/mnist',train=False,download=True,transform=transform)
test_loader = DataLoader(test_dataset,shuffle=False,batch_size=batch_size)

## 定义模型

### Inception模块的定义

In [4]:
# 改变的是channel大小
class InceptionA(torch.nn.Module):
    def __init__(self,in_channels):
        super(InceptionA,self).__init__()
        self.branch1x1 = torch.nn.Conv2d(in_channels,16,kernel_size=1)
        
        self.branch5x5_1 = torch.nn.Conv2d(in_channels,16,kernel_size=1)
        self.branch5x5_2 = torch.nn.Conv2d(16,24,kernel_size=5,padding=2)# padding=2才能保持原有图像大小不变
        
        self.branch3x3_1 = torch.nn.Conv2d(in_channels,16,kernel_size=1)
        self.branch3x3_2 = torch.nn.Conv2d(16,24,kernel_size=3,padding=1)
        self.branch3x3_3 = torch.nn.Conv2d(24,24,kernel_size=3,padding=1)
        
        self.branch_pool = torch.nn.Conv2d(in_channels,24,kernel_size=1)
        
    def forward(self,x):
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)
        branch3x3 = self.branch3x3_3(branch3x3)
        
        branch_pool = F.avg_pool2d(x,kernel_size=3,stride=1,padding=1)
        branch_pool = self.branch_pool(branch_pool)
        
        # 将输出结果进行拼接
        outputs = [branch1x1,branch5x5,branch3x3,branch_pool]
        # dim(batch,channel,width,height)，我们要将channel拼接在一起
        return torch.cat(outputs,dim=1)

### 定义网络模型

In [6]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # 先定义两个卷积层
        self.conv1 = torch.nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = torch.nn.Conv2d(88,20,kernel_size=5)
        
        self.incep1 = InceptionA(in_channels=10)
        self.incep2 = InceptionA(in_channels=20)
        
        # 经过Maxpooling之后，图像的高度和宽度是一直在减小的
        self.mp = torch.nn.MaxPool2d(2)
        # 1408是算出来的
        self.fc = torch.nn.Linear(1408,10)
        
    def forward(self,x):
        in_size = x.size(0)
        # 先做一次卷积，然后做一次池化，最后做激活
        x = F.relu(self.mp(self.conv1(x))) # 做完这个，输入通道变成10个
        x = self.incep1(x) # 经过计算，最后输出通道是88个
        x = F.relu(self.mp(self.conv2(x))) # 把输入88个通道降成20个通道
        x = self.incep2(x) # 最后又输出88个通道
        x = x.view(in_size,-1) 
        x = self.fc(x)
        return x
        
    
model = Net()

## 构造损失函数和优化器

In [7]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01,momentum=0.5)

## 构造训练器

In [8]:
def train(epoch):
    running_loss = 0.0
    for batch_idx,data in enumerate(train_loader,0):
        inputs,target = data
        # 把inputs,target都迁移到GPU中
        # inputs,target = inputs.to(device),target.to(device)
        # 优化器在优化之前先清零
        optimizer.zero_grad()
        # forward backward update
        outputs = model(inputs)
        loss = criterion(outputs,target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        # 每300输出一次
        if batch_idx % 300 == 299:
            print('[%d,%5d] loss:%.3f' % (epoch+1,batch_idx+1,running_loss/300))
            # 每300一组数据跑完，清零
            running_loss = 0.0

## 测试

In [9]:
# test里面不需要计算反向传播，只需要计算正向的就行了
def test():
    # 表示正确了多少
    correct = 0
    # 表示总数有多少
    total = 0
    # 在test过程中是不需要计算梯度的
    # 这部分的代码不会求导
    with torch.no_grad():
        for data in test_loader:
            images,labels = data
            # 把数据迁移到GPU上
            # images,labels = images.to(device),labels.to(device)
            outputs = model(images)
            # dim=1 返回每一个行的最大值和索引
            # dim=0 返回每一列的最大值和索引
            # 这里的索引或者下标对应的是它的分类
            _,predicted = torch.max(outputs.data,dim=1)
            # label是一个元组，(N,1)
            # size是计算元素的个数，0表示计算行数，1表示计算列数
            total += labels.size(0)
            correct += (predicted==labels).sum().item()
            
    # 使用正确的数除以总数
    print('Accuracy on test set: %d %%' % (100 * correct/total))

In [10]:
for epoch in range(10):
    # 一轮训练
    train(epoch)
    # 一轮测试
    test()

[1,  300] loss:0.992
[1,  600] loss:0.213
[1,  900] loss:0.131
Accuracy on test set: 96 %
[2,  300] loss:0.108
[2,  600] loss:0.092
[2,  900] loss:0.088
Accuracy on test set: 98 %
[3,  300] loss:0.072
[3,  600] loss:0.073
[3,  900] loss:0.064
Accuracy on test set: 98 %
[4,  300] loss:0.058
[4,  600] loss:0.057
[4,  900] loss:0.058
Accuracy on test set: 98 %
[5,  300] loss:0.051
[5,  600] loss:0.050
[5,  900] loss:0.048
Accuracy on test set: 98 %
[6,  300] loss:0.046
[6,  600] loss:0.042
[6,  900] loss:0.045
Accuracy on test set: 98 %
[7,  300] loss:0.038
[7,  600] loss:0.039
[7,  900] loss:0.041
Accuracy on test set: 98 %
[8,  300] loss:0.033
[8,  600] loss:0.040
[8,  900] loss:0.036
Accuracy on test set: 98 %
[9,  300] loss:0.034
[9,  600] loss:0.033
[9,  900] loss:0.031
Accuracy on test set: 98 %
[10,  300] loss:0.029
[10,  600] loss:0.033
[10,  900] loss:0.028
Accuracy on test set: 98 %


## 总结
第二轮就达到98%,效果还是杠杠的